<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/Project/retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install langchain
!pip install tiktoken
!pip install langchain-openai
!pip install faiss-cpu
!pip install langchain-community

In [24]:
import os
from pathlib import Path
import subprocess
import shutil
from langchain.text_splitter import TokenTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

from google.colab import userdata

In [25]:
def clone_git_repository(repo_url, clone_dir):
    """
    주어진 GitHub 저장소를 지정된 디렉토리에 클론합니다.
    """
    if os.path.exists(clone_dir):
        print(f"Directory {clone_dir} already exists. Deleting and recloning the repository.")
        shutil.rmtree(clone_dir)
    subprocess.run(['git', 'clone', repo_url, clone_dir], check=True)
    print(f"Repository cloned to {clone_dir}")

def read_code_files(directory, extensions=['.py']):
    """
    지정된 디렉토리에서 주어진 확장자를 가진 모든 코드 파일을 읽어서 반환합니다.
    """
    code_texts = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(file.endswith(ext) for ext in extensions):
                file_path = Path(root) / file
                with open(file_path, 'r', encoding='utf-8') as f:
                    code_texts.append(f.read())
    return code_texts

def split_text_with_overlap(text, chunk_size=1000, overlap=200):
    """
    TokenTextSplitter를 사용하여 텍스트를 오버랩을 가진 청크로 나눕니다.
    """
    text_splitter = TokenTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        disallowed_special=()  # 모든 특수 토큰을 허용하지 않음
    )
    return text_splitter.split_text(text)

def process_code_directory(directory, extensions=['.py', '.cpp', '.h'], chunk_size=1000, overlap=200):
    """
    디렉토리 내의 모든 코드 파일을 읽고, TokenTextSplitter를 사용하여 오버랩이 있는 청크로 나눠서 반환합니다.
    """
    code_texts = read_code_files(directory, extensions)
    chunks = []
    for text in code_texts:
        chunks.extend(split_text_with_overlap(text, chunk_size, overlap))
    return chunks

In [29]:
# def main():
repo_url = 'https://github.com/ggerganov/llama.cpp.git'
clone_dir = './llama_cpp'
openai_organization_id = userdata.get('MY_OPENAI_ORG_ID')
openai_key = userdata.get('OPEN-AI_KEY')

# GitHub 저장소 클론
# clone_git_repository(repo_url, clone_dir)

# 클론한 디렉토리 내의 코드 파일 처리
chunks = process_code_directory(clone_dir, extensions=['.py', '.cpp', '.h'], chunk_size=1000, overlap=200)

# 임시코드
chunks = chunks[:30]

# 청크 출력
# for i, chunk in enumerate(chunks):
#     print(f"Chunk {i+1}:")
#     print(chunk)
#     print("\n" + "-"*80 + "\n")

# VectorStore를 생성합니다.
embeddings = OpenAIEmbeddings(openai_api_key=openai_key, organization=openai_organization_id)
vector = FAISS.from_texts(chunks, embeddings)

# if __name__ == "__main__":
#     main()


Chunk 1:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# This script downloads the tokenizer models of the specified models from Huggingface and
# generates the get_vocab_base_pre() function for convert_hf_to_gguf.py
#
# This is necessary in order to analyze the type of pre-tokenizer used by the model and
# provide the necessary information to llama.cpp via the GGUF header in order to implement
# the same pre-tokenizer.
#
# ref: https://github.com/ggerganov/llama.cpp/pull/6920
#
# Instructions:
#
# - Add a new model to the "models" list
# - Run the script with your huggingface token:
#
#   python3 convert_hf_to_gguf_update.py <huggingface_token>
#
# - Copy-paste the generated get_vocab_base_pre() function into convert_hf_to_gguf.py
# - Update llama.cpp with the new pre-tokenizer if necessary
#
# TODO: generate tokenizer tests for llama.cpp
#

import logging
import os
import pathlib
import re

import requests
import sys
import json

from hashlib import sha256
from enum import IntEnum,

In [40]:
# Retriever를 생성합니다.
base_retriever = vector.as_retriever(
    search_type='similarity_score_threshold',
    search_kwargs={'score_threshold': 0.08}
)


from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI


model_name = "gpt-3.5-turbo"
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN-AI_KEY')

llm=ChatOpenAI(model=model_name, temperature=0.4, organization=openai_organization_id, max_tokens=500)

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever,
    llm=llm
)

In [41]:
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever

# langchain 패키지의 tools 모듈에서 retriever 도구를 생성
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=multi_query_retriever
)

In [42]:
from langchain.tools.retriever import create_retriever_tool

# langchain 패키지의 tools 모듈에서 retriever 도구를 생성
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=multi_query_retriever
)

retriever_tool = create_retriever_tool(
    compression_retriever,
    name="pdf_search",
    # 도구에 대한 설명을 자세히 기입해야 합니다!!!
    description="2023년 12월 AI 관련 정보를 PDF 문서에서 검색합니다. '2023년 12월 AI 산업동향' 과 관련된 질문은 이 도구를 사용해야 합니다!",
)

AIMessage(content='`modify_tensors` 함수는 텐서를 수정하는 데 사용되는 함수입니다. 이 함수는 텐서의 값을 변경하거나 모양을 조정하는 데 사용될 수 있습니다. \n\n`modify_tensors` 함수를 사용하는 방법은 다음과 같습니다:\n\n1. 필요한 모듈을 import 합니다.\n```python\nimport tensorflow as tf\n```\n\n2. 텐서를 생성합니다.\n```python\ntensor = tf.constant([[1, 2, 3], [4, 5, 6]])\n```\n\n3. `modify_tensors` 함수를 사용하여 텐서를 수정합니다. 예를 들어, 텐서의 값을 변경하거나 모양을 조정할 수 있습니다.\n```python\nmodified_tensor = tf.add(tensor, 10) # 텐서의 모든 요소에 10을 더합니다.\n```\n\n4. 수정된 텐서를 출력하거나 다른 작업에 사용할 수 있습니다.\n```python\nprint(modified_tensor)\n```\n\n위의 예시에서는 `tf.add` 함수를 사용하여 텐서의 모든 요소에 10을 더하는 방법을 보여주었습니다. `modify_tensors` 함수를 사용하여 다양한 작업을 수행할 수 있으며, 필요에 따라 다른 함수를 사용하여 텐서를 수정할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 333, 'prompt_tokens': 20, 'total_tokens': 353}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1fdf4a24-e209-44cb-9e9d-e2ac51b3d9a5-0', usage_metadata={'input_token

In [47]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿 작성
prompt_template = """
다음 질문에 대한 관련 정보를 검색하고 답변을 제공합니다:

질문: {question}

관련 정보: {retrieved_content}

답변을 간결하고 명확하게 작성하세요.
"""

# 프롬프트를 생성합니다.
prompt = PromptTemplate(
    input_variables=["question", "retrieved_content"],
    template=prompt_template,
)

# LLMChain 생성
qa_chain = LLMChain(
    llm=llm,
    prompt=prompt,
)

def get_answer_with_retrieval(question):
    # 리트리버를 사용해 관련 문서를 검색합니다.
    retrieved_docs = compression_retriever.get_relevant_documents(question)

    # 검색된 내용을 텍스트로 결합합니다.
    retrieved_content = "\n".join([doc.page_content for doc in retrieved_docs])
    return retrieved_content
    # 검색된 정보를 포함한 프롬프트로 LLM에 전달하여 답변을 생성합니다.
    # answer = qa_chain.run({
    #     "question": question,
    #     "retrieved_content": retrieved_content,
    # })

    # return answer

# 예시 질문
question = "SentencePieceTokenTypes class 초기값"
answer = get_answer_with_retrieval(question)

print("Question:", question)
print("Answer:", answer)

Question: SentencePieceTokenTypes class 초기값
Answer: SentencePieceTokenTypes class 초기값
